In [1]:
import numpy as np

import openbabel

from pymatgen import Molecule
from pymatgen.core.operations import SymmOp
from pymatgen.util.coord_utils import get_angle
from pymatgen.io.babel import BabelMolAdaptor

In [2]:
class Polymer(object):
    def __init__(self, start_monomer, s_head, s_tail, 
                 monomer, head, tail, 
                 end_monomer, e_head, e_tail, 
                 n_units, link_distance=1.0, linear_chain=False):
        """
        Args:
            monomer (Molecule): The monomer
            n_units (int): number of monomer units excluding the start and terminal molecules
            head (int): index of the atom in the monomer that forms the head
            tail (int): tail atom index. monomers will be connected from head to tail
            link_distance (float): distance between consecutive monomers
        """
        self.start = s_head
        self.end = s_tail
        self.monomer = monomer
        self.n_units = n_units
        self.link_distance = link_distance
        #translate monomers so that head atom is at the origin
        start_monomer.translate_sites(range(len(start_monomer)), - monomer.cart_coords[s_head])
        monomer.translate_sites(range(len(monomer)), - monomer.cart_coords[head])
        end_monomer.translate_sites(range(len(end_monomer)), - monomer.cart_coords[e_head])
        self.mon_vector = monomer.cart_coords[tail] - monomer.cart_coords[head]
        self.moves = {1:[1,0,0], 2:[0,1,0], 3:[0,0,1], 4:[-1,0,0], 5:[0,-1,0], 6:[0,0,-1]}
        self.prev_move = 1
        #places the start monomer at the beginning of the chain
        self.molecule = start_monomer.copy()
        self.length = 0
        # create the chain
        self._create(self.monomer, self.mon_vector, linear_chain)
        # terminate the chain with the end_monomer
        self.n_units += 1
        end_mon_vector = end_monomer.cart_coords[e_tail] - end_monomer.cart_coords[e_head]
        self._create(end_monomer, end_mon_vector, linear_chain)
    
    def _create(self, monomer, mon_vector, linear_chain):
        """
        create the polymer from the monomer
        """
        while self.length != self.n_units:
            if linear_chain:
                move_direction = np.array(mon_vector)/np.linalg.norm(mon_vector)
            else:
                move_direction = self._next_move_direction()
            self._add_monomer(monomer.copy(), mon_vector, move_direction)
            
    def _next_move_direction(self):
        """
        pick a move at random from the list of moves
        """
        move = np.random.randint(1, 7)
        while self.prev_move == (move + 3)%6:
            move = np.random.randint(1, 7)
        self.prev_move = move
        print "move", move
        return np.array(self.moves[move])
    
    def _align_monomer(self, monomer, mon_vector, move_direction):
        """
        rotate the monomer so that it is aligned along the move direction
        """
        axis = np.cross(mon_vector, move_direction)
        origin = monomer[self.start].coords
        angle = get_angle(mon_vector, move_direction)
        #print "axis, origin, angle", axis, origin, angle
        op = SymmOp.from_origin_axis_angle(origin, axis, angle)
        monomer.apply_operation(op)

    def _add_monomer(self, monomer, mon_vector, move_direction):
        """
        extend the polymer molecule by adding a monomer along mon_vector direction
        
        Args:
            monomer (Molecule): monomer molecule
            mon_vector (numpy.array): monomer vector that points from head to tail.
            move_direction (numpy.array): direction along which the monomer will be positioned
        """
        translate_by = self.molecule.cart_coords[self.end] + self.link_distance * move_direction
        monomer.translate_sites(range(len(monomer)), translate_by)
        self._align_monomer(monomer, mon_vector, move_direction)
        # add monomer if there are no crossings
        does_cross = False
        for i, site in enumerate(monomer):
            try:
                self.molecule.append(site.specie, site.coords, properties=site.properties)
            except:
                does_cross = True
                polymer_length = len(self.molecule)
                self.molecule.remove_sites(range(polymer_length-i, polymer_length))
                break
        if not does_cross:
            self.length += 1
            self.end += len(self.monomer)
            print "length: ", self.length

Get the molecular topology info using openbabel

In [3]:
def get_topology(molecule):
    bma = BabelMolAdaptor(molecule)
    obmol = bma.openbabel_mol
    #print obmol.NumAtoms(), obmol.NumBonds()
    atoms = [x.GetIdx()-1 for x in openbabel.OBMolAtomIter(obmol)]
    bonds = [[x.GetBeginAtomIdx()-1, x.GetEndAtomIdx()-1] for x in openbabel.OBMolBondIter(obmol)]
    angles = [list(x) for x in openbabel.OBMolAngleIter(obmol)]
    dihedrals = [list(x) for x in openbabel.OBMolTorsionIter(obmol)]
    #print len(atoms), len(bonds), len(angles), len(dihedrals)
    atoms = [tuple([str(molecule[x].specie), 
                    molecule[x].ff_map]) for x in atoms ]
    bonds = [x+[tuple(sorted((molecule[x[0]].ff_map, 
                              molecule[x[1]].ff_map)))] for x in bonds ]
    angles = [x+[tuple(((molecule[x[0]].ff_map, 
                         molecule[x[1]].ff_map, 
                         molecule[x[2]].ff_map)))] for x in angles ]
    dihedrals =  [x+[tuple(sorted((molecule[x[0]].ff_map, 
                                   molecule[x[1]].ff_map, 
                                   molecule[x[2]].ff_map, 
                                   molecule[x[3]].ff_map)))] for x in dihedrals ]
    return atoms, bonds, angles, dihedrals

Set the start, bulk and terminal molecules. Also set the forcefield species name mapping

In [4]:
# start molecule
peo_start = Molecule.from_file("PEOmonomer_start.xyz")
ff_map = ["Ce", "H", "H", "H", "O", "Cm", "H", "H"]
peo_start.add_site_property("ff_map", ff_map)
s_head = 0
s_tail = 5

# chain molecule
peo_bulk = Molecule.from_file("PEOmonomer_bulk.xyz")
ff_map = ["Ce", "H", "H", "O", "Cm", "H", "H"]
peo_bulk.add_site_property("ff_map", ff_map)
head = 0
tail = 4

# terminal molecule
peo_end = Molecule.from_file("PEOmonomer_end.xyz")
ff_map = ["Ce", "H", "H", "O", "Cm", "H", "H", "H"]
peo_end.add_site_property("ff_map", ff_map)
e_head = 0
e_tail = 4

Linear polymer chain

In [5]:
n_units = 3
link_distance = 3.0

# create the polymer
peo_polymer = Polymer(peo_start, s_head, s_tail, 
                      peo_bulk, head, tail, 
                      peo_end, e_head, e_tail, 
                      n_units, link_distance, linear_chain=True)

peo_polymer.molecule.to(filename="polymer_linear.xyz", fmt="xyz")

length:  1
length:  2
length:  3
length:  4


Get the topology of the linear chain

In [6]:
atoms, bonds, angles, dihedrals = get_topology(peo_polymer.molecule)

Print unique bonds, angles and dihedrals in the linear chain

In [7]:
print set([x[2] for x in bonds])
print set([x[3] for x in angles])
print set([x[4] for x in dihedrals])

set([('Ce', 'H'), ('Ce', 'O'), ('Cm', 'H'), ('Cm', 'O')])
set([('Cm', 'O', 'H'), ('Cm', 'H', 'H'), ('O', 'Ce', 'Cm'), ('Ce', 'H', 'O'), ('Ce', 'H', 'H')])
set([('Ce', 'Cm', 'H', 'O')])


Random walk polymer chain

In [8]:
n_units = 10
link_distance = 3.0

# create the polymer
peo_polymer = Polymer(peo_start, s_head, s_tail, 
                      peo_bulk, head, tail, 
                      peo_end, e_head, e_tail, 
                      n_units, link_distance)

peo_polymer.molecule.to(filename="polymer.xyz", fmt="xyz")

move 6
length:  1
move 5
length:  2
move 3
length:  3
move 5
length:  4
move 6
length:  5
move 3
move 1
length:  6
move 2
length:  7
move 6
length:  8
move 6
length:  9
move 3
move 3
move 5
length:  10
move 1
length:  11


Use openbabel to optimize the polymer chain.
Local optimization is used to preserve the chain structure.

In [9]:
peo_obabel = BabelMolAdaptor(peo_polymer.molecule)
# ['gaff', 'ghemical', 'mmff94', 'mmff94s', 'uff']
peo_obabel.localopt(forcefield='mmff94', steps=500)
peo_optimized = peo_obabel.pymatgen_mol

Add the site property "ff_map" to the openbabel optimized polymer chain

In [10]:
ff_map = [site.ff_map for site in peo_polymer.molecule]
peo_optimized.add_site_property("ff_map", ff_map)
peo_optimized.to(filename="polymer_optimized.xyz", fmt="xyz")

Get the polymer topology: atoms, bonds, angles and dihedrals

In [11]:
atoms, bonds, angles, dihedrals = get_topology(peo_optimized)

print len(atoms), len(bonds), len(angles), len(dihedrals)

86 74 90 50


Use Packmol to pack the optimized polymer chains into a box

In [12]:
from rubicon.io.packmol.packmol import PackmolRunner 

packmol_config = [{"number": 1, "fixed": [0, 0, 0, 0, 0, 0],"centerofmass": ""},
                  {"number": 30, "inside sphere": [0, 0, 0, 200.0]}]
pmr = PackmolRunner([peo_optimized, peo_optimized],
                    packmol_config,
                    tolerance=2.0,
                    filetype="xyz",
                    control_params={"nloop": 1000},
                    auto_box=False, output_file="poly_packed.xyz")
packed_polymer = pmr.run()

packed molecule written to poly_packed.xyz
